# Baseline
### Базовые модели на основе цены закрытия (трасформированной до логарифмического прироста):
- Наивный прогноз
- Логистическая регрессия


In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
import Handlers as hd

import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_validate, GridSearchCV
import optuna

from sklearn import set_config
set_config(transform_output = 'pandas')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_palette('Set2')

# Загрузка данных
используется файл обработанных данных, в котором:
- удален начальный, нерелевантный, период
- обработаны пропуски
- произведена трансформация признаков
- сгенерированны новые признаки

### Оставляю в качестве признака только Log_Return и целевой признак

In [3]:
TARGET = "Long"
df = pd.read_parquet(
    "../data/ETH-Full-1H_prepared.parquet", columns=["Close_log_return", TARGET]
)
df = df.asfreq("H")  # установка периода для timeseries

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 45120 entries, 2018-03-01 00:00:00 to 2023-04-23 23:00:00
Freq: H
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Close_log_return  45120 non-null  float64
 1   Long              45120 non-null  int32  
dtypes: float64(1), int32(1)
memory usage: 881.2 KB


# Transform TimeSeries to Dataset for Supervised Learning
Определяется глубина последовательности данных T, которая будет использоваться для построения прогноза. Временное окно в прошлое.

Выбираю 6 часовое окно.

In [5]:
T = 6

Create $X$, $y$

In [6]:
X, y, N, D = hd.create_X_y_from_timeseries(df, TARGET, T)

In [7]:
print("X.shape:", X.shape, "y.shape:", y.shape, "N:", N, "D:", D)

X.shape: (45114, 6) y.shape: (45114,) N: 45114 D: 1


# Split Data
Разбиение с стратифиацией

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42, test_size=0.3
)

# Scaling Data

В данном случае, т.к. один признак используется для baseline автокорреляционной модели, то в скалировании нет необходимости.

В дальнейшем, нужно скалировать признаки, приводить их к одному масштабу. Путем перебора выбирать лучший скалер.

In [9]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Наивный прогноз
### Просто предсказание значения классом большинства

In [10]:
y_train_pred = np.zeros(len(X_train))
y_test_pred = np.zeros(len(X_test))

hd.print_classification_metrics(y_train, y_train_pred, y_test, y_test_pred)

*** TRAIN ***
Accuracy: 0.910
Precision: 0.000
Recall: 0.000
F1: 0.000
ROC AUC: 0.500

*** TEST ***
Accuracy: 0.910
Precision: 0.000
Recall: 0.000
F1: 0.000
ROC AUC: 0.500


Если смотреть по Accuracy, то вполне ок ) Предсказаны все объекты класса 0.

А все остальные метрики говорят, что модель плохая.

А значение ROC AUC = 0.5 говорит, что модель работает на уровне случайного угадывания.

# Логистическая регрессия
### С гиперпараметрами по умолчанию

In [11]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

hd.print_classification_metrics(y_train, y_train_pred, y_test, y_test_pred)

*** TRAIN ***
Accuracy: 0.910
Precision: 0.000
Recall: 0.000
F1: 0.000
ROC AUC: 0.500

*** TEST ***
Accuracy: 0.910
Precision: 0.000
Recall: 0.000
F1: 0.000
ROC AUC: 0.500


Из-за несбалансированности классов все ушло в предсказание класса 0. 

Результат такой же как и для наивного прогноза.

# Логистическая регрессия
### С взвешиванием классов

In [12]:
model = LogisticRegression(class_weight="balanced")
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

hd.print_classification_metrics(y_train, y_train_pred, y_test, y_test_pred)

*** TRAIN ***
Accuracy: 0.530
Precision: 0.101
Recall: 0.533
F1: 0.170
ROC AUC: 0.531

*** TEST ***
Accuracy: 0.531
Precision: 0.104
Recall: 0.548
F1: 0.174
ROC AUC: 0.539


Результат чуть лучше чем случайный. Возьмем его за Baseline.

### Добавляю выбор скалера в пайплайн через кастомный трансформер, где конкретный тип скалера выбирается в рамках GridSearchCV

In [23]:
pipe = Pipeline(
    [
        ("scaler", hd.SelectScaler(name="standart")),
        ("classifier", LogisticRegression(max_iter=10000, class_weight="balanced")),
    ]
)

param_grid = {
    "scaler__name": ["standard", "minmax", "robust", "mean_normalization", "no_scaler"],
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(pipe, param_grid, cv=skf, scoring="f1", n_jobs=-1)
grid_search.fit(X_train, y_train)

y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)
hd.print_classification_metrics(y_train, y_train_pred, y_test, y_test_pred)

Best parameters: {'scaler__name': 'standard'}
Best score: 0.16731642517896658
*** TRAIN ***
Accuracy: 0.530
Precision: 0.101
Recall: 0.534
F1: 0.170
ROC AUC: 0.532

*** TEST ***
Accuracy: 0.530
Precision: 0.103
Recall: 0.545
F1: 0.173
ROC AUC: 0.537


Precision на тестовой выборке еще немного вырос c 0.096 до 0.098

### Использование Optuna для подбора гиперпараметров

In [ ]:
def objective(trial, X=X_train, y=y_train):
    penalty_solver_combo = trial.suggest_categorical(
        "penalty_solver",
        [
            ("l1", "liblinear"),
            ("l1", "saga"),
            ("l2", "newton-cg"),
            ("l2", "lbfgs"),
            ("l2", "liblinear"),
            ("l2", "sag"),
            ("l2", "saga"),
            ("none", "newton-cg"),
            ("none", "lbfgs"),
            ("none", "sag"),
            ("none", "saga"),
        ],
    )
    penalty = penalty_solver_combo[0]
    solver = penalty_solver_combo[1]

    C = trial.suggest_float("classifier__C", 0.01, 1)
    max_iter = trial.suggest_int("classifier__max_iter", 50, 5000)

    name = trial.suggest_categorical(
        "scaler__name",
        ["standard", "minmax", "robust", "mean_normalization", "no_scaler"],
    )

    pipe = Pipeline(
        [
            ("scaler", hd.SelectScaler(name=name)),
            (
                "classifier",
                LogisticRegression(
                    C=C,
                    penalty=penalty,
                    solver=solver,
                    max_iter=max_iter,
                    class_weight="balanced",
                ),
            ),
        ]
    )

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_metrics = cross_validate(
        estimator=pipe,
        X=X,
        y=y,
        cv=skf,
        scoring="f1",
        n_jobs=-1,
        return_train_score=False,
    )

    score = np.mean(cv_metrics["test_score"])

    return score

In [ ]:
%%time
study = optuna.create_study(study_name='LogisticRegression', direction="maximize")
study.optimize(objective, n_trials=10)

print('Best f1 value:', study.best_value)
print('Best params:', study.best_params)

[I 2023-08-30 15:42:38,947] A new study created in memory with name: LogisticRegression
[I 2023-08-30 15:42:41,554] Trial 0 finished with value: 0.16772873191800888 and parameters: {'penalty_solver': ('l2', 'saga'), 'classifier__C': 0.48664718674767365, 'classifier__max_iter': 2248, 'scaler__name': 'no_scaler'}. Best is trial 0 with value: 0.16772873191800888.
[I 2023-08-30 15:42:46,769] Trial 1 finished with value: 0.1697172644099565 and parameters: {'penalty_solver': ('l2', 'saga'), 'classifier__C': 0.5029569509069058, 'classifier__max_iter': 4480, 'scaler__name': 'standard'}. Best is trial 1 with value: 0.1697172644099565.
[I 2023-08-30 15:42:47,406] Trial 2 finished with value: 0.16940205311284118 and parameters: {'penalty_solver': ('none', 'lbfgs'), 'classifier__C': 0.7800046272205047, 'classifier__max_iter': 3198, 'scaler__name': 'minmax'}. Best is trial 1 with value: 0.1697172644099565.
[I 2023-08-30 15:46:11,394] Trial 3 finished with value: 0.1668771167597812 and parameters: {

Best f1 value: 0.1697172644099565
Best params: {'penalty_solver': ('l2', 'saga'), 'classifier__C': 0.5029569509069058, 'classifier__max_iter': 4480, 'scaler__name': 'standard'}
CPU times: total: 203 ms
Wall time: 3min 36s


In [ ]:
study.optimize(objective, n_trials=20)

print('Best f1 value:', study.best_value)
print('Best params:', study.best_params)

[I 2023-08-30 16:05:15,763] Trial 50 finished with value: 0.16846738669540814 and parameters: {'penalty_solver': ('l2', 'sag'), 'classifier__C': 0.16571074661921179, 'classifier__max_iter': 2317, 'scaler__name': 'mean_normalization'}. Best is trial 48 with value: 0.16995007601651255.
[I 2023-08-30 16:05:16,417] Trial 51 finished with value: 0.16981766730331777 and parameters: {'penalty_solver': ('none', 'lbfgs'), 'classifier__C': 0.10170544083151214, 'classifier__max_iter': 2604, 'scaler__name': 'robust'}. Best is trial 48 with value: 0.16995007601651255.
[I 2023-08-30 16:05:17,089] Trial 52 finished with value: 0.16969655617104823 and parameters: {'penalty_solver': ('l2', 'lbfgs'), 'classifier__C': 0.049511411546401374, 'classifier__max_iter': 3137, 'scaler__name': 'robust'}. Best is trial 48 with value: 0.16995007601651255.
[I 2023-08-30 16:06:03,325] Trial 53 finished with value: 0.16943931663095194 and parameters: {'penalty_solver': ('l2', 'sag'), 'classifier__C': 0.095774976975449

Best f1 value: 0.17047227893148664
Best params: {'penalty_solver': ('l2', 'sag'), 'classifier__C': 0.05704761580609944, 'classifier__max_iter': 2668, 'scaler__name': 'robust'}


In [ ]:
best_params = study.best_params.copy()

if "penalty_solver" in best_params:
    penalty, solver = best_params["penalty_solver"]
    best_params["classifier__penalty"] = penalty
    best_params["classifier__solver"] = solver
    del best_params["penalty_solver"]

In [ ]:
best_params
# Best f1 value: 0.26191126592125363
# {'classifier__C': 0.5404716578189352,
#  'classifier__max_iter': 3390,
#  'scaler__name': 'minmax',
#  'classifier__penalty': 'l2',
#  'classifier__solver': 'sag'}

{'classifier__C': 0.05704761580609944,
 'classifier__max_iter': 2668,
 'scaler__name': 'robust',
 'classifier__penalty': 'l2',
 'classifier__solver': 'sag'}

In [9]:
best_params = {'classifier__C': 0.05704761580609944,
 'classifier__max_iter': 2668,
 'scaler__name': 'robust',
 'classifier__penalty': 'l2',
 'classifier__solver': 'sag'}

In [10]:
pipe = Pipeline(
    [
        ("scaler", hd.SelectScaler(name="no_scaler")),
        (
            "classifier",
            LogisticRegression(
                class_weight="balanced",
            ),
        ),
    ]
)

In [11]:
# Set the parameters in the pipeline
pipe.set_params(**best_params)

Pipeline(steps=[('scaler', SelectScaler(name='robust')),
                ('classifier',
                 LogisticRegression(C=0.05704761580609944,
                                    class_weight='balanced', max_iter=2668,
                                    solver='sag'))])

In [12]:
# Обучаем модель
pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

hd.print_classification_metrics(y_train, y_train_pred, y_test, y_test_pred)

*** TRAIN ***
Accuracy: 0.090
Precision: 0.090
Recall: 1.000
F1: 0.166
ROC AUC: 0.500

*** TEST ***
Accuracy: 0.090
Precision: 0.090
Recall: 1.000
F1: 0.165
ROC AUC: 0.500


Значение метрик стали немного лучше, чем при простом поиске по сетке с кросс-валидацией